In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
!pip install numpy==1.21.6
!pip install sentence-transformers
!pip install faiss-cpu
!pip install langid
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.9/15.9 MB 48.4 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: numpy
    Found existing installation: numpy 1.23.5
    Uninstalling numpy-1.23.5:
      Successfully uninstalled numpy-1.23.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
apache-beam 2.46.0 requires dill<0.3.2,>=0.3.1.1, but you have dill 0.3.6 which is incompatible.
momepy 0.6.0 requires shapely>=2, but you have shapely 1.8.5.post1 which is incompatible.
pymc3 3.11.5 requires scipy<1.8.0,>=1.7.3, but you have scipy 1.10.1 which is incompatible.
tensorflow 2.12.0 requires numpy<1.24,>=1.22, but you have numpy 1.21.6 which is incompatible.
ydata-profiling 4.1.2 requires scipy<1.10,>=1.4.1, but you have scipy 1.10.1 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 4.1 MB/s eta 0:00:00
  Preparing metadata (setup.p

In [3]:
import xml.sax
import csv
import random
import faiss
import numpy as np
from sentence_transformers import SentenceTransformer

In [4]:
from datasets import load_dataset

dataset = load_dataset("imdb")

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

Dataset imdb downloaded and prepared to /root/.cache/huggingface/datasets/imdb/plain_text/1.0.0/2fdd8b9bcadd6e7055e742a706876ba43f19faee861df134affd7a3f60fc38a1. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [5]:
import csv
from datasets import load_dataset

# Load the IMDB dataset
dataset = load_dataset("imdb")
train_dataset = dataset["train"]

csv_file = "imdb_train_dataset.csv"

# Open the CSV file in write mode
with open(csv_file, mode="w", encoding="utf-8", newline="") as file:
    writer = csv.writer(file)

    writer.writerow(["text", "label"])

    for example in train_dataset:
        text = example["text"]
        label = example["label"]
        writer.writerow([text, label])

print("Dataset saved to:", csv_file)

  0%|          | 0/3 [00:00<?, ?it/s]

Dataset saved to: imdb_train_dataset.csv


In [6]:
# Load the IMDB dataset
dataset = load_dataset("imdb")
test_dataset = dataset["test"]  # Use the "test" split instead of "train"

csv_file = "imdb_test_dataset.csv"  # Use a different CSV file name for the test dataset

# Open the CSV file in write mode
with open(csv_file, mode="w", encoding="utf-8", newline="") as file:
    writer = csv.writer(file)

    writer.writerow(["text", "label"])

    for example in test_dataset:  # Iterate over the test dataset instead of train dataset
        text = example["text"]
        label = example["label"]
        writer.writerow([text, label])

print("Test dataset saved to:", csv_file)

  0%|          | 0/3 [00:00<?, ?it/s]

Test dataset saved to: imdb_test_dataset.csv


In [7]:
# Load the training data from the train CSV file
train_texts = []
train_labels = []
with open("imdb_train_dataset.csv", mode="r", encoding="utf-8") as file:
    reader = csv.reader(file)
    next(reader)  # Skip the header row
    for row in reader:
        train_texts.append(row[0])
        train_labels.append(row[1])

# Load the test data from the test CSV file
test_texts = []
test_labels = []
with open("imdb_test_dataset.csv", mode="r", encoding="utf-8") as file:
    reader = csv.reader(file)
    next(reader)  # Skip the header row
    for row in reader:
        test_texts.append(row[0])
        test_labels.append(row[1])


In [8]:
model = SentenceTransformer("nlptown/bert-base-multilingual-uncased-sentiment")

/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZTVN10tenso

In [9]:
# Generate text embeddings for the training data
batch_size = 128
train_embeddings = []

for i in range(0, len(train_texts), batch_size):
    batch_texts = train_texts[i : i + batch_size]
    batch_embeddings = model.encode(batch_texts)
    train_embeddings.extend(batch_embeddings)

train_embeddings = np.array(train_embeddings)

Batches:   0%|          | 0/4 [00:00<?, ?it/s]

Batches:   0%|          | 0/4 [00:00<?, ?it/s]

Batches:   0%|          | 0/4 [00:00<?, ?it/s]

Batches:   0%|          | 0/4 [00:00<?, ?it/s]

Batches:   0%|          | 0/4 [00:00<?, ?it/s]

...

Batches:   0%|          | 0/4 [00:00<?, ?it/s]

Batches:   0%|          | 0/4 [00:00<?, ?it/s]

Batches:   0%|          | 0/4 [00:00<?, ?it/s]

Batches:   0%|          | 0/4 [00:00<?, ?it/s]

Batches:   0%|          | 0/2 [00:00<?, ?it/s]

In [10]:
# Generate text embeddings for the test data
test_embeddings = []

for i in range(0, len(test_texts), batch_size):
    batch_texts = test_texts[i : i + batch_size]
    batch_embeddings = model.encode(batch_texts)
    test_embeddings.extend(batch_embeddings)

test_embeddings = np.array(test_embeddings)

Batches:   0%|          | 0/4 [00:00<?, ?it/s]

Batches:   0%|          | 0/4 [00:00<?, ?it/s]

Batches:   0%|          | 0/4 [00:00<?, ?it/s]

Batches:   0%|          | 0/4 [00:00<?, ?it/s]

Batches:   0%|          | 0/4 [00:00<?, ?it/s]

...

Batches:   0%|          | 0/4 [00:00<?, ?it/s]

Batches:   0%|          | 0/4 [00:00<?, ?it/s]

Batches:   0%|          | 0/4 [00:00<?, ?it/s]

Batches:   0%|          | 0/4 [00:00<?, ?it/s]

Batches:   0%|          | 0/2 [00:00<?, ?it/s]

In [11]:
# For train vectors 
train_vectors_file = "train_vectors.npy"
test_vectors_file = "test_vectors.npy"
np.save(train_vectors_file, train_embeddings)
np.save(test_vectors_file, test_embeddings)
print("Train vectors saved to:", train_vectors_file)
print("Test vectors saved to:", test_vectors_file)

Train vectors saved to: train_vectors.npy
Test vectors saved to: test_vectors.npy


In [12]:
# For test vectors
train_labels_file = "train_labels.npy"
test_labels_file = "test_labels.npy"
np.save(train_labels_file, train_labels)
np.save(test_labels_file, test_labels)
print("Train labels saved to:", train_labels_file)
print("Test labels saved to:", test_labels_file)

Train labels saved to: train_labels.npy
Test labels saved to: test_labels.npy


In [13]:
# Build an index with the training text embeddings
index = faiss.IndexFlatL2(train_embeddings.shape[1])
index.add(train_embeddings)

In [14]:
# Define a query example
query_example = "test" # This is a test query 

# Encode the query example
query_embedding = model.encode([query_example])[0]

# Perform a k-nearest neighbors search
k = 11
distances, indices = index.search(query_embedding.reshape(1, -1), k)

# Print the nearest neighbors and their labels
print("Nearest neighbors:")
for i, idx in enumerate(indices[0]):
    neighbor_text = train_texts[idx]
    neighbor_label = train_labels[idx]
    print(f"Neighbor {i+1}: {neighbor_text} | Label: {neighbor_label}")

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Nearest neighbors:
Neighbor 1: Rudy does it again with this hot off the streets follow up to Dolemite. This entry is filled with the requisite Rudy Ray Moore raunch, humor and martial arts. Rudy eludes a crazy red-neck sheriff in this movie that also features an infamous scene where Rudy dives down a steep hill. See it for laughs and for a brain-blasting hit of Blaxploitation magic. | Label: 1
Neighbor 2: Story of Ireland in the 70/s. This film is a beautiful reconstruction of small time Ireland in the 1970/s. All the gang are there see below. Master Boyle , The Boys , The Cannon , SP O'Donnell , Senator Doogan's Daugter , Rose , Agnes , Maura and Una. See this film.Feel Ireland as it was.<br /><br /> | Label: 1
Neighbor 3: This movie is utterly hilarious. Its cast clicks immediately with frame one and takes us on a wonderful ride through spoofing gangster films. The conflict of brother vs. brother appears when Johnny's brother becomes a do-gooder D.A. However, the best character is Jo

In [15]:
import json
# Define a query example
k = 11  # Number of nearest neighbors to retrieve
results = []  # List to store the results

for i in range(len(test_embeddings)):
    query_embedding = test_embeddings[i]
    query_label = test_labels[i]

    # Perform a k-nearest neighbors search
    distances, indices = index.search(query_embedding.reshape(1, -1), k)

    # Get the labels of the nearest neighbors
    neighbor_labels = [train_labels[idx] for idx in indices[0]]

    # Find the proposed label based on majority vote
    proposed_label = max(set(neighbor_labels), key=neighbor_labels.count)

    # Create a dictionary to store the results
    result = {
        "text": test_texts[i],
        "proposed_label": proposed_label,
        "topKSimilarTexts": []
    }

    # Add the top k similar texts and their labels
    for j, idx in enumerate(indices[0]):
        neighbor_text = train_texts[idx]
        neighbor_label = train_labels[idx]
        result["topKSimilarTexts"].append({"text": neighbor_text, "label": neighbor_label})

    # Add the result to the results list
    results.append(result)

# Save the results in JSON format
results_file = "test_results.json"
with open(results_file, "w") as file:
    json.dump(results, file, indent=4)

print("Test results saved to:", results_file)

Test results saved to: test_results.json


In [18]:
import csv
import json

# Load the test data from the CSV file
test_data = []
with open("imdb_test_dataset.csv", mode="r", encoding="utf-8") as file:
    reader = csv.reader(file)
    next(reader)  # Skip the header row
    test_data = list(reader)

# Load the test results from the JSON file
with open("test_results.json", "r") as file:
    test_results = json.load(file)

# Create a dictionary to store the actual labels
actual_labels = {}
for row in test_data:
    text = row[0]
    label = row[1]
    actual_labels[text] = label

# Compare the proposed labels with the actual labels and calculate accuracy
correct_predictions = 0
total_predictions = len(test_results)

for result in test_results:
    text = result["text"]
    proposed_label = result["proposed_label"]
    actual_label = actual_labels[text]

    if proposed_label == actual_label:
        correct_predictions += 1

accuracy = (correct_predictions / total_predictions) * 100

print("Accuracy:", accuracy)

Accuracy: 90.62400000000001
